# Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:\
\
-->page_content: a string representing the content\
\
-->metadata: a dict containing arbritary metadata. The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

In [9]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions known for their loyalty and friendliness.",
        metadata = {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata = {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata = {"source":"fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata = {"source":"bird-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata = {"source":"mammal-pets-doc"}
    )
]

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [12]:
#Vector Store objects themselves are not direct runnables
#But their retriever interface is runnable
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [13]:
vectorstore.similarity_search("cat")

[Document(id='fc2f2434-d9d7-4dd6-935c-3439ce3c495a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b5c8c284-7a21-4872-95d0-71238cdca843', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='0a5dc223-4175-4058-aec9-ecef25fc2318', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.'),
 Document(id='f203f3d2-f5c6-48e4-ab7b-c5633064337c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]

In [14]:
## async query
await vectorstore.asimilarity_search("cat")

[Document(id='fc2f2434-d9d7-4dd6-935c-3439ce3c495a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b5c8c284-7a21-4872-95d0-71238cdca843', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='0a5dc223-4175-4058-aec9-ecef25fc2318', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.'),
 Document(id='f203f3d2-f5c6-48e4-ab7b-c5633064337c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]

# Retreivers
LangChain vectorstore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains

LangChain Retrievers are Runnables, so they implement a standard set of methods (like synchronous and asynchronous inoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to retrieve documents, we can create a runnable easily. 



In [15]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

#we can convert Vector Store into Retriever by using RunnableLambda
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) #bind k=1 gives top result
retriever.batch(["cat","dog"])


[[Document(id='fc2f2434-d9d7-4dd6-935c-3439ce3c495a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='f203f3d2-f5c6-48e4-ab7b-c5633064337c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]]

Vectorstores implement as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following

In [16]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs = {"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='fc2f2434-d9d7-4dd6-935c-3439ce3c495a', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='f203f3d2-f5c6-48e4-ab7b-c5633064337c', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness.')]]

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("tell me about dogs")
print(response.content)

According to the provided context, dogs are great companions known for their loyalty and friendliness.


What we implemented in this code is a form of RAG (Retrival-Augmented Generation) where a retriever is used to fetch relevant context (like documents) from a vector store, and this context is then passed to a LLM to generate a response.